# MSBA7011 Homework 1

## Q1: MNIST  3-layer neural network

In [1]:
# Importing
import numpy as np
import tensorflow as tf

In [2]:
# Load the data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("DATA/MNIST/", one_hot=True)

Extracting DATA/MNIST/train-images-idx3-ubyte.gz
Extracting DATA/MNIST/train-labels-idx1-ubyte.gz
Extracting DATA/MNIST/t10k-images-idx3-ubyte.gz
Extracting DATA/MNIST/t10k-labels-idx1-ubyte.gz


In [3]:
# Get training and testing data set
train_x = mnist.train.images
train_y = mnist.train.labels

test_x = mnist.test.images
test_y = mnist.test.labels

# Define some parameters
num_cls = 10
num_features = 28 * 28

In [4]:
# TensorFlow Setup

# Some parameters
num_iter = 10000
batch_size = 100
num_hidden_1 = 256
num_hidden_2 = 128

# Placeholder variables
x = tf.placeholder(dtype=tf.float32, shape=[None, num_features]) 
y_true = tf.placeholder(dtype=tf.float32, shape=[None, num_cls]) 

# Input -- Hidden 1
W1 = tf.Variable(tf.truncated_normal(shape=[num_features, num_hidden_1], stddev=0.06))
b1 = tf.Variable(tf.constant(0.1,shape=[num_hidden_1]))
# Hidden 1 -- Hidden 2
W2 = tf.Variable(tf.truncated_normal(shape=[num_hidden_1, num_hidden_2], stddev=0.06))
b2 = tf.Variable(tf.constant(0.1,shape=[num_hidden_2]))
# Hidden 2 -- Output
W3 = tf.Variable(tf.truncated_normal(shape=[num_hidden_2, num_cls], stddev=0.06))
b3 = tf.Variable(tf.zeros(shape=[num_cls]))


# Model
h1 = tf.nn.relu(tf.matmul(x, W1) + b1)
h2 = tf.nn.relu(tf.matmul(h1, W2) + b2)
logits = tf.matmul(h2, W3) + b3
y_pred = tf.nn.softmax(logits) # to know prob distribution

# Cost function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=logits)) # the function avoids log(0) situation

# Optimization
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)

In [5]:
# TensorFlow Training

# Session
sess = tf.InteractiveSession()

# Initialize variables
tf.global_variables_initializer().run()

# Run TensorFlow with Stochastic Gradient Descent
for ik in range(num_iter):
  batch_x, batch_y = mnist.train.next_batch(batch_size)
  sess.run(optimizer, feed_dict={x: batch_x, y_true: batch_y})

# for ik in range(num_iter):
#     sess.run(optimizer, feed_dict={x: train_x, y_true: train_y})

In [6]:
# Evaluation

results_pred = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
accuracy = tf.reduce_mean(tf.cast(results_pred, tf.float32))
print("Accuracy: ", sess.run(accuracy, feed_dict={x: test_x, y_true: test_y}))

# Around 98%

Accuracy:  0.979


In [7]:
# Close Session
sess.close()

## Q2: Iris 4-layer neural network

In [11]:
# Draw things on the Jupyter Notebook
%matplotlib inline  

# Importing
import numpy as np
import tensorflow as tf
import pandas as pd
import os
from six.moves.urllib.request import urlopen

In [12]:
# Data sets
IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

# If the training and test sets aren't stored locally, download them.
if not os.path.exists(IRIS_TRAINING):
  raw = urlopen(IRIS_TRAINING_URL).read()
  with open(IRIS_TRAINING,'wb') as f:
    f.write(raw)

if not os.path.exists(IRIS_TEST):
  raw = urlopen(IRIS_TEST_URL).read()
  with open(IRIS_TEST,'wb') as f:
    f.write(raw)
    
# Load datasets.
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TRAINING,
    target_dtype=np.int,
    features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TEST,
    target_dtype=np.int,
    features_dtype=np.float32)

### Load Iris Dataset into Python

use pandas to load the Iris data into Python. 

In [13]:
# Load datasets via Pandas.
filename_iris_train = "iris_training.csv"
filename_iris_test = "iris_test.csv"

iris_train = pd.read_csv(filename_iris_train)
iris_test = pd.read_csv(filename_iris_test)

# Convert to One-hot encoding
iris_train_onehot = pd.get_dummies(iris_train,columns=['virginica'])
iris_test_onehot = pd.get_dummies(iris_test,columns=['virginica'])

# Training data
iris_train_x = iris_train_onehot.values[:,:-3]
iris_train_y = iris_train_onehot.values[:,-3:]
# Test data
iris_test_x = iris_test_onehot.values[:,:-3]
iris_test_y = iris_test_onehot.values[:,-3:]

## Explore the Data

In [14]:
# Explore the dataset
print(iris_train_x.shape)
print(iris_train_y.shape)

print(iris_test_x.shape)
print(iris_test_y.shape)

(120, 4)
(120, 3)
(30, 4)
(30, 3)


In [15]:
# Based on the above outputs, define the following parameters
num_cls = 3
num_features = 4

## Modeling via Deep Neural Network

In [16]:
# TensorFlow Setup

# Some parameters
num_iter = 5000
# Number of hidden units in the first hidden layer
num_h1 = 10
# Number of hidden units in the second hidden layer
num_h2 = 20
# Number of hidden units in the third hidden layer
num_h3 = 10

# Placeholder variables
x = tf.placeholder(dtype=tf.float32, shape=[None, num_features]) 
y_true = tf.placeholder(dtype=tf.float32, shape=[None, num_cls]) 

# Input -- Hidden 1
W1 = tf.Variable(tf.truncated_normal(shape=[num_features, num_h1], stddev=0.06))
b1 = tf.Variable(tf.constant(0.1,shape=[num_h1]))
# Hidden 1 -- Hidden 2
W2 = tf.Variable(tf.truncated_normal(shape=[num_h1, num_h2], stddev=0.06))
b2 = tf.Variable(tf.constant(0.1,shape=[num_h2]))
# Hidden 2 -- Hidden 3
W3 = tf.Variable(tf.truncated_normal(shape=[num_h2, num_h3], stddev=0.06))
b3 = tf.Variable(tf.constant(0.1,shape=[num_h3]))
# Hidden 3 -- Output
W4 = tf.Variable(tf.truncated_normal(shape=[num_h3, num_cls], stddev=0.06))
b4 = tf.Variable(tf.zeros(shape=[num_cls]))


# Model
h1 = tf.nn.relu(tf.matmul(x, W1) + b1)
h2 = tf.nn.relu(tf.matmul(h1, W2) + b2)
h3 = tf.nn.relu(tf.matmul(h2, W3) + b3)
logits = tf.matmul(h3, W4) + b4
y_pred = tf.nn.softmax(logits) # to know prob distribution


# Cost function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=logits))

# Optimization
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)

In [17]:
# TensorFlow Train

# Session
sess = tf.InteractiveSession()

# Initialize variables
tf.global_variables_initializer().run()

# Run TensorFlow
for ik in range(num_iter):
    sess.run(optimizer, feed_dict={x: iris_train_x, y_true: iris_train_y})

In [18]:
# Evaluation

results_pred = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
accuracy = tf.reduce_mean(tf.cast(results_pred, tf.float32))
print("Accuracy: ", sess.run(accuracy, feed_dict={x: iris_test_x, y_true: iris_test_y}))

Accuracy:  0.966667


In [19]:
# Close Session
sess.close()